# Thursday 13 Nov 25 - Day 4

# Python 2 HSUTCC: Session 8: Decorator

In [ ]:
from collections.abc import Callable
import functools

# Callable is a type hint that means 'something you can call like a function'


## What is a decorator?

Decorator is a function that recieve a function and then return something. For example,

In [ ]:
def my_decorator(target_func: Callable) -> int:
    return 20

# target_function is a parameter
# When u call my_decorator(foo), now target_function holds the function object foo.

In [ ]:
def foo() -> None:
    print('I am silly.')

# putting foo inside my_decorator as an argument
bar = my_decorator(foo) # passing the function
print(bar)
# it won't print I am silly. cause foo is now 20 (not a function anymore)

bar = my_decorator(foo()) 
# run foo() first, print "I am silly.", then pass None into the decorator
print(bar)

Now, to use a decorator, we have a shorthand or as people normally call **Syntactic Sugar** using `@` sign as follow.

In [ ]:
@my_decorator
def foo() -> None:
    print('I am silly.')

foo()

# equivalent to foo = my_decorator(foo)
# Because my_decorator(foo) returns 20, now foo == 20
# so foo() is 20()


In [ ]:
print(foo)

What happend? How come our `foo` function become an integer? Well, the thing that happend behind the hood is

In [ ]:
# function object -> <1234>
# foo -> <1234>
def foo() -> None:
    print('I am silly.')

# foo = 20
foo = my_decorator(foo)

print(foo)

In [ ]:
@my_decorator
def foo() -> None:
    print(2 + 4)
    print('I am silly.')

print(foo)

## Heck! What are we talking about?

### Adding functionalities, the naked way.

Let's take a step back and imagine some scenario.

Say you are working on a project where you have a send notification function implemented where, by default you send the notification via email to a list of contact.

In [ ]:
contact = ['Ve', 'Due', 'Batman', 'Superman']

In [ ]:
def send_noti(contact: list) -> None:
    for each_contact in contact:
        print(f'Send email to {each_contact}.')

In [ ]:
send_noti(contact)

Now, what happend if you want to send the notification via SMS as well?

In [ ]:
def send_noti(contact: list, via_sms: bool=False) -> None:
    for each_contact in contact:
        print(f'Send email to {each_contact}.')

    if via_sms:
        for each_contact in contact:
            print(f'Send SMS to {each_contact}.')

In [ ]:
send_noti(contact, via_sms=True)

Now, if we add more channels, our code will become something like

In [ ]:
def send_noti(
                contact: list,
                via_sms: bool=False,
                via_fb: bool=False,
                via_slack: bool=False
              ) -> None:

    for each_contact in contact:
        print(f'Send email to {each_contact}.')

    if via_sms:
        for each_contact in contact:
            print(f'Send SMS to {each_contact}.')

    if via_fb:
        for each_contact in contact:
            print(f'Send SMS to {each_contact}.')

    if via_slack:
        for each_contact in contact:
            print(f'Send SMS to {each_contact}.')

This is one way that we can add more functionalities to our function (we start with a function that send notification via email and then we add SMS, Facebook, and Slack to the functionality). With this concept of adding functionality on top of each other, we have another method - using decorator.

### Adding functionalities, the decorated way.

Let's take a look at the normal structure of decorator (wrapper).

In [ ]:
def wrapper(my_func):

    def inner_func(*args, **kwargs):
        # Do something useful
        pass

    return inner_func

Again, begining with our base `send_noti` function.

In [ ]:
def send_noti(contact: list) -> None:
    for each_contact in contact:
        print(f'Send email to {each_contact}.')

To add the SMS functionality, we will create a new decorator.

In [ ]:
def sms_decorator(func: Callable[[list], None]) -> None:
    def inner(contact):
        for each_contact in contact:
            print(f'Send SMS to {each_contact}.')

        func(contact)

    return inner

In [ ]:
# @sms_decorator
# def send_noti(contact: list) -> None: # <123>
#     for each_contact in contact:
#         print(f'Send email to {each_contact}.')

# send_noti = sms_decorator(send_noti)

# def sms_decorator(func): # <123>
#     def inner(contact): # <124>
#         for each_contact in contact:
#             print(f'Send SMS to {each_contact}.')

#         func(contact) # <123>

#     return inner # <124>

# send_noti # <124>

In [ ]:
@sms_decorator
def send_noti(contact: list) -> None:
    for each_contact in contact:
        print(f'Send email to {each_contact}.')

send_noti(contact)

# @sms_decorator
# def send_noti(contact):
#     ...
# = send_noti = sms_decorator(send_noti)

# That means:
# - send_noti gets replaced by whatever sms_decorator returns.
# - sms_decorator returns the function inner.
# - So after decoration, send_noti becomes inner, not the original send_noti.

# send_noti(contact) becomes inner(contact)

# call send_noti → actually calls inner → inner sends SMS → inner calls send_noti → send_noti sends email


Voilà, seem like our code work correctly. Let's stop for a moment and disect what going on.

``` python
def send_noti(contact: list) -> None:
    for each_contact in contact:
        print(f'Send email to {each_contact}.')
```

``` python
send_noti = sms_decorator(send_noti)
```

What happend here?

``` python
def inner(contact):
    for each_contact in contact:
        print(f'Send SMS to {each_contact}.')
    
    send_noti(contact)
```

``` python
send_noti = inner
```

Now, the `send_noti` becomes

``` python
def send_noti(contact):
    for each_contact in contact:
        print(f'Send SMS to {each_contact}.')
    
    # the base send_noti
    send_noti(contact)
```

``` python
send_noti(contact)
```

Okayyyyy, take a deep breathe and process what just happend. We always have time.

Ready? Now, to add more functionalities, we can do the same.

In [ ]:
def fb_decorator(func: Callable[[list], None]) -> None:
    def inner(contact):
        for each_contact in contact:
            print(f'Send FB to {each_contact}.')

        func(contact)

    return inner

def slack_decorator(func: Callable[[list], None]) -> None:
    def inner(contact):
        for each_contact in contact:
            print(f'Send Slack to {each_contact}.')

        func(contact)

    return inner

In [ ]:
@sms_decorator
@fb_decorator
@slack_decorator
def send_noti(contact: list) -> None:
    for each_contact in contact:
        print(f'Send email to {each_contact}.')

send_noti(contact)

#### Sidenote: Function Documentation

Suppose that we have a documentation for our `send_noti` function.

In [ ]:
def send_noti(contact: list) -> None:
    """Send notification to contacts"""
    for each_contact in contact:
        print(f'Send email to {each_contact}.')

If you check the documentation by hovering over the function or using the magic command `?`, you will see the documentation.

In [ ]:
?send_noti

In [ ]:
help(send_noti)

Now, let's add the decorator and see what happend.

In [ ]:
def sms_decorator(func: Callable[[list], None]) -> None:
    def inner(contact):
        # """This documentation is from the decorator!"""
        for each_contact in contact:
            print(f'Send SMS to {each_contact}.')

        func(contact)

    return inner

In [ ]:
@sms_decorator
def send_noti(contact: list) -> None:
    """Send notification to contacts"""
    for each_contact in contact:
        print(f'Send email to {each_contact}.')

In [ ]:
?send_noti

The documentation is gone! This is not desirable at all. To fix this, we will need a `functools` module.

In [ ]:
def sms_decorator(func: Callable[[list], None]) -> None:

    @functools.wraps(func)
    def inner():
        for each_contact in contact:
            print(f'Send SMS to {each_contact}.')

        func(contact)

    return inner

# after adding decorator with  @functools.wraps(func)
@sms_decorator
def send_noti(contact: list) -> None:
    """Send notification to contacts"""
    for each_contact in contact:
        print(f'Send email to {each_contact}.')

print(send_noti.__name__)
print(send_noti.__doc__)

In [ ]:
def sms_decorator(func: Callable[[list], None]) -> None:

    # @functools.wraps(func)
    def inner():
        for each_contact in contact:
            print(f'Send SMS to {each_contact}.')

        func(contact)

    return inner

# after adding decorator without  @functools.wraps(func)
@sms_decorator
def send_noti(contact: list) -> None:
    """Send notification to contacts"""
    for each_contact in contact:
        print(f'Send email to {each_contact}.')

print(send_noti.__name__)
print(send_noti.__doc__)

In [ ]:
?send_noti

In [ ]:
# before adding decorator
def send_noti(contact: list) -> None:
    """Send notification to contacts"""
    for each_contact in contact:
        print(f'Send email to {each_contact}.')

print(send_noti.__name__)
print(send_noti.__doc__)

All is good now.

## Some examples

### @timethis

In [ ]:
import time

def timethis(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, end=" ... ")

        start_time = time.time()
        result = func(*args, **kwargs)
        elapse_time = time.time() - start_time

        print(elapse_time)
        return result
    return inner

'''timethis is a decorator.
Inside it, there is another function called inner.
This inner function:

prints the function name

records the time before running the function

runs the original function

calculates and prints how long it took

returns the result of the original function'''


Original function → add
Wrapper function → inner
(The wrapper REPLACES the original function (add).
The wrapper calls the original function inside itself.
The wrapper is the new function that the original becomes.)
Analogy
The original function = the gift
The wrapper function = the wrapping paper
After decorating, what you see is the wrapping paper, not the gift **wraps makes the wrapper function IDENTIFY as the original, but it does not change the fact that the wrapper’s behavior is different.** 
(add now points to inner, inner still calls the original add inside itself
wraps just copies metadata from the original to the wrapper)
But the gift is still inside, and you can open it (call it) when needed

After decoration:
add  →  inner (wrapper)
inner → calls original add

then:
functools.wraps(original_add)  
    ↓  
copies attributes  
    ↓  
to inner (wrapper)

so:
inner "pretends" to be add
but still calls original add


*args and **kwargs let inner accept any arguments…
…but they do not change what the original function can accept.
The original function must support those arguments.

summary:
inner replaces add, and wraps makes inner look like the original add (name, docstring, etc.).

In [ ]:
@timethis
def add(a, b):
    return a + b

add(2, 5)

'''
@timethis
def add(a, b):
    return a + b

means add = timethis(add)
So now, add does not refer to the original add function anymore.
Instead, add refers to the inner function returned by the decorator.

But because we used @functools.wraps(func), the new add looks like the old one (same name, same docstring).

when we call add(2, 5), python actually call inner(2, 5)
Inside inner:
It prints the function's name (add ...)
It starts a timer

It calls the real function:
result = func(2, 5)  # func is the original add

It prints how long it took

It returns the real result (7)
'''

In [ ]:
@timethis
def factorial(number):
    result = 1
    
    for current_num in range(1, number + 1):
        result *= current_num

    return result

factorial(1000)

In [ ]:
import time

# A bigger version
def timethis(func=None, *, n_iter=100):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, end=" ... ")

        acc = float("inf")
        for _ in range(n_iter):
            tick = time.perf_counter()
            result = func(*args, **kwargs)
            acc = min(acc, time.perf_counter() - tick)
        print(acc)

        return result
    return inner

result = timethis(sum)(range(10 ** 6))
print(result)

### @once

In [ ]:
def once(func):

    @functools.wraps(func)
    def inner(*args, **kwargs):

        if not inner.called:
            func(*args, **kwargs)
            inner.called = True

    inner.called = False

    return inner

In [ ]:
@once
def print_3():
    print(3)

for _ in range(1000):
    print_3()

In [ ]:
def once_v2(func):

    called = False # local variable in once_v2

    @functools.wraps(func)
    def inner(*args, **kwargs):
        nonlocal called # refers to called in the outer function

        if not called:
            func(*args, **kwargs)
            called = True
    
    print(locals()) # prints local variables in once_v2

    return inner

In [ ]:
@once_v2
def print_3():
    print(3)

for _ in range(1000):
    print_3()

In [ ]:
print(globals())

### @memoized

In [ ]:
def memoized(func):
    cache = {}

    @functools.wraps(func)
    def inner(*args, **kwargs):
        key = args + tuple(sorted(kwargs.items()))
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]

    return inner

@memoized
def ackermann(m, n):
    if not m:
        return n + 1
    elif not n:
        return ackermann(m - 1, 1)
    else:
        return ackermann(m - 1, ackermann(m, n - 1))

ackermann(3, 4)
ackermann(3, 4)

## Decorator with arguments

What if we want to add some arguments for the decorator? For example, initializing the origin address for SMS messaging.

In [ ]:
def sms_decorator(func: Callable[[list], None], origin: str) -> None:

    @functools.wraps(func)
    def inner(contact):
        for each_contact in contact:
            print(f'Send SMS from {origin} to {each_contact}.')

        func(contact)

    return inner

@sms_decorator('Salmon3748')
def send_noti(contact: list) -> None:
    for each_contact in contact:
        print(f'Send email to {each_contact}.')

send_noti(contact)

If we do that directly in the decorator definition, we wouldn't be able to go to sleep (since the code will break). In order to do that, we will introduce a new layer of decorator - `with_arguments`.

In [ ]:
def with_arguments(deco):
    @functools.wraps(deco)
    def wrapper(*dargs, **dkwargs):

        @functools.wraps(wrapper)
        def decorator(func):
            result = deco(func, *dargs, **dkwargs)
            return result

        return decorator
    return wrapper

Let's see it in action.

In [ ]:
# sms_decorator = with_arguments(sms_decorator) # <123>

# def with_arguments(deco):
#     @functools.wraps(deco)
#     def wrapper(*dargs, **dkwargs): # <124>

#         @functools.wraps(wrapper)
#         def decorator(func): # <125>
#             result = deco(func, *dargs, **dkwargs)
#             return result

#         return decorator
#     return wrapper

# # sms_decorator <124>

# @sms_decorator('Salmon3748')
# def send_noti(contact: list) -> None:
#     for each_contact in contact:
#         print(f'Send email to {each_contact}.')

# sms_decorator = sms_decorator('Salmon3748') # <124>
# # sms_decorator <125>
# # send_noti <126>

# send_noti = sms_decorator(send_noti)
# # deco(func, *dargs, **dkwargs) <123>
# # <123>(<126>, 'Salmon3748')
# def sms_decorator(func: Callable[[list], None], origin: str) -> None:

#     @functools.wraps(func)
#     def inner(contact):
#         for each_contact in contact:
#             print(f'Send SMS from {origin} to {each_contact}.')

#         func(contact)

#     return inner

In [ ]:
@with_arguments
def sms_decorator(func: Callable[[list], None], origin: str) -> None:

    @functools.wraps(func)
    def inner(contact):
        for each_contact in contact:
            print(f'Send SMS from {origin} to {each_contact}.')

        func(contact)

    return inner

@sms_decorator('Salmon3748')
def send_noti(contact: list) -> None:
    for each_contact in contact:
        print(f'Send email to {each_contact}.')

send_noti(contact)

### Sidenote: Code Breakdown

``` python
@with_arguments
def sms_decorator(func, origin):

    @functools.wraps(func)
    def inner(contact):
        for each_contact in contact:
            print(f'Send SMS from {origin} to {each_contact}.')
        
        func(contact)
    
    return inner
```

The normal decorator call

``` python
sms_decorator = with_arguments(sms_decorator)
```

The function wrapper is created.

``` python
def wrapper(*dargs, **dkwargs):
    
    @functools.wraps(wrapper)
    def decorator(func):
        result = sms_decorator(func, *dargs, **dkwargs)
        return result
    
    return decorator

sms_decorator = wrapper
```

Now, the decorator is used.

``` python
@sms_decorator('Salmon3748')
def send_noti(contact):
    for each_contact in contact:
        print(f'Send email to {each_contact}.')
```

Initializing a decorator with arguments is a two-step process, first, the arguments are passed into the `wrapper` function and `decorator` function is defined.

``` python
deco = sms_decorator('Salmon3748')

def decorator(func):
    result = sms_decorator(func, 'Salmon3748')
    return result
```

Then the decorator is passed in the function as usual.

``` python
send_noti = deco(send_noti)

result = sms_decorator(send_noti, 'Salmon3748')
```

And the process continue...

## Decorators with optional arguments

To create an optional argument is now easy enough.

In [ ]:
@with_arguments
def sms_decorator(func: Callable[[list], None], origin: str='Salmon3748') -> None:

    @functools.wraps(func)
    def inner(contact):
        for each_contact in contact:
            print(f'Send SMS from {origin} to {each_contact}.')

        func(contact)

    return inner

@sms_decorator()
def send_noti(contact: list) -> None:
    for each_contact in contact:
        print(f'Send email to {each_contact}.')

send_noti(contact)

In [ ]:
# Function Namespace
# Module Namespace
# Class Namespace

name = 'Luna'

class Cat:
    name = 'Oscar'

cat = Cat()
cat.name

# Tasks (18 November 2025)

1. Write a decorator `n_times` which given a number `n` makes a function to execute `n` times.

```python
@n_times(3)
def print_my_name():
    print('Due')

print_my_name()
```

```terminal
Due
Due
Due
```

In [ ]:
# Your code here
def n_times(n_times: int) -> Callable:  # 1. This receives the number n
    '''
    Decorator to execute a function n times.

    Args:
        n_times (int): Number of times to execute the function.

    Returns:
        Callable: Decorated function executed n times.
    '''
    
    def decorator(func: Callable) -> Callable: # 2. This receives the target function
        '''
        Receives the target function to be decorated.

        Args:
            func (Callable): The target function to decorate.

        Returns:
            Callable: Wrapper function that executes func n times.
        '''

        def wrapper(*args, **kwargs) -> str: # 3. This calls the original function n times
            '''
            Wrapper function that calls the original function n times.

            Args:
                *args: Positional arguments passed to the original function.
                **kwargs: Keyword arguments passed to the original function.

            Returns:
                str: Return value of the original function. 
                For this task, the original function prints the name(str).
    
            '''

            for _ in range(n_times):
                func(*args, **kwargs)
        return wrapper
    return decorator

@n_times(5)
def print_my_name():
    ''' print the name 'Due' '''
    print('Due')

print_my_name()


2. Write a decorator `once` which makes a function to execute only once and other times it just returns the result calculated on the first call.

```python
@once
def random_number():
    return random.randint(1, 10)

print(random_number())
print(random_number())
print(random_number())
```

```terminal
4
4
4
```

In [ ]:
# Your code here
import functools
import random

def once(func: Callable) -> Callable:
    '''
    Decorator to ensure a function executes only once. 
    Subsequent calls return the first result.
    
    Args:
        func (Callable): The target function to decorate.

    Returns:
        Callable: Decorated function that runs only once.
    '''

    @functools.wraps(func)

    def wrapper(*args, **kwargs) -> int:
        '''
        Wrapper function that runs the original function only once, stores the result, and returns it for subsequent calls.

        Args:
            *args: Positional arguments passed to the original function.
            **kwargs: Keyword arguments passed to the original function.

        Returns:
            int: Result of the original function (same every call).
                For this task, the original function returns an int.

        Notes:
        Uses internal attributes wrapper.called and wrapper.result to store state between calls.
        '''

        if not wrapper.called:
            wrapper.result = func(*args, **kwargs)  # store first result
            wrapper.called = True
        return wrapper.result

    wrapper.called = False   # initial state
    wrapper.result = None    # initial number
    return wrapper


@once
def random_number():
    '''
    Returns a random integer between 1 and 10 (inclusive).

    This function is decorated with @once, so it will only generate a random number the first time it is called.
    Subsequent calls return the same number.
    
    Returns:
        int: A random integer between 1 and 10.
    '''
    return random.randint(1, 10) # Returns a random integer between 1 & 10 inclusive.

print(random_number())
print(random_number())
print(random_number())

7
7
7
